<a href="https://colab.research.google.com/github/Frenchyy1/test/blob/master/Copie_de_Classify_Airbnb_Reviews_with_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference
- https://towardsdatascience.com/nlp-extract-contextualized-word-embeddings-from-bert-keras-tf-67ef29f60a7b
- https://github.com/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb

# Prepare

In [0]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import os
import json
import sys
from datetime import datetime
import tensorflow as tf
from google.colab import files

pd.options.display.max_columns = 100

In [0]:
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
    print('TPU devices:')
    display(session.list_devices())

    # Upload credentials to TPU.
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.117.190.154:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:


[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 17535882111920883680),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10358072035438471755),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 11563146933754790441),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 9363616543302077282),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 15158598008150000435),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 16080320278391696299),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 16870948931959813101),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 15782709768404034606),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 1209286891455019171),
 _DeviceAttributes(/job:tpu_worker/repli

In [0]:
!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
    sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

Cloning into 'bert_repo'...
remote: Enumerating objects: 336, done.
remote: Total 336 (delta 0), reused 0 (delta 0), pack-reused 336
Receiving objects: 100% (336/336), 283.23 KiB | 3.88 MiB/s, done.
Resolving deltas: 100% (185/185), done.



# Data Preprocessing

In [0]:
TASK = 'SST-2'
DATA_NAME = 'SST'

# Download glue data.
! test -d download_glue_repo || git clone https://gist.github.com/60c2bdb54d156a41194446737ce03e2e.git download_glue_repo
!python download_glue_repo/download_glue_data.py --data_dir='glue_data' --tasks=$DATA_NAME

TASK_DATA_DIR = 'glue_data/' + TASK
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))
!ls $TASK_DATA_DIR

BUCKET = 'springboard-project-nlp'
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert-capstone-project/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

df_train = pd.read_csv(TASK_DATA_DIR + '/train.tsv', delimiter='\t')
print(len(df_train))
print(df_train.head())

Cloning into 'download_glue_repo'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21
Unpacking objects: 100% (21/21), done.
	Completed!
***** Task data directory: glue_data/SST-2 *****
dev.tsv  original  test.tsv  train.tsv
***** Model output directory: gs://springboard-project-nlp/bert-capstone-project/models/SST-2 *****
67349
                                            sentence  label
0       hide new secretions from the parental units       0
1               contains no wit , only labored gags       0
2  that loves its characters and communicates som...      1
3  remains utterly satisfied to remain the same t...      0
4  on the worst revenge-of-the-nerds clichés the ...      0


In [0]:
# The data is modified to a suitable form for a prediction.
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'label'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

df_train = pd.read_csv(TASK_DATA_DIR + '/train.tsv', delimiter='\t')
train_InputExamples = df_train.apply(
    lambda x: run_classifier.InputExample(
        guid=None, # Globally unique ID for bookkeeping, unused in this example
        text_a = x[DATA_COLUMN], 
        text_b = None, 
        label = x[LABEL_COLUMN]
    ), axis = 1
)

df_dev = pd.read_csv(TASK_DATA_DIR + '/dev.tsv', delimiter='\t')
dev_InputExamples = df_dev.apply(
    lambda x: run_classifier.InputExample(
        guid=None, # Globally unique ID for bookkeeping, unused in this example
        text_a = x[DATA_COLUMN], 
        text_b = None, 
        label = x[LABEL_COLUMN]
    ), axis = 1
)

display(len(train_InputExamples))
display(train_InputExamples.head())

67349

0    <run_classifier.InputExample object at 0x7ff62...
1    <run_classifier.InputExample object at 0x7ff62...
2    <run_classifier.InputExample object at 0x7ff62...
3    <run_classifier.InputExample object at 0x7ff62...
4    <run_classifier.InputExample object at 0x7ff62...
dtype: object

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df_reviews = pd.read_csv('drive/My Drive/Springboard/projects/data/reviews.csv')

display(len(df_reviews))
display(df_reviews.columns)
display(df_reviews.head())

298447

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,35303,810980,2011-12-28,1502908,Firuz,Miyuki's has been a great host. Helpful and ve...
1,35303,2467161,2012-10-01,350719,Jordan,Nice apartment in one of the best areas of Tok...
2,35303,3572628,2013-02-18,4917704,Aymeric,A very nice host and place. The location is in...
3,35303,3952097,2013-03-30,3243253,Blandine,Miyuki is the most smily face I met when I lan...
4,35303,4377509,2013-05-01,1536097,Kayleigh,The location's wonderful. It's close to Haraju...


In [0]:
df_test = pd.DataFrame({
    'id': df_reviews.id,
    'comments': df_reviews.comments
})
df_test = df_test.dropna()
table = str.maketrans({'\t': '', '\n': '', '\r': ''})
df_test['comments'] = df_test['comments'].map(lambda x:x.translate(table))

test_InputExamples = df_test.apply(
  lambda x: run_classifier.InputExample(
    guid = x.id,
    text_a = x['comments'], 
    text_b = None, 
    label = 0
  ), axis = 1
)

display(len(test_InputExamples))
display(df_test.head())
display(test_InputExamples.head())

298277

,id,comments
0,810980,Miyuki's has been a great host. Helpful and ve...
1,2467161,Nice apartment in one of the best areas of Tok...
2,3572628,A very nice host and place. The location is in...
3,3952097,Miyuki is the most smily face I met when I lan...
4,4377509,The location's wonderful. It's close to Haraju...


0    <run_classifier.InputExample object at 0x7ff61...
1    <run_classifier.InputExample object at 0x7ff61...
2    <run_classifier.InputExample object at 0x7ff61...
3    <run_classifier.InputExample object at 0x7ff61...
4    <run_classifier.InputExample object at 0x7ff61...
dtype: object

In [0]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

# Convert our train and dev features to InputFeatures that BERT understands.
train_features = run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
dev_features = run_classifier.convert_examples_to_features(dev_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 67349


INFO:tensorflow:Writing example 0 of 67349


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Writing example 270000 of 298277


INFO:tensorflow:Writing example 280000 of 298277


INFO:tensorflow:Writing example 280000 of 298277


INFO:tensorflow:Writing example 290000 of 298277


INFO:tensorflow:Writing example 290000 of 298277


# Creating a model

In [0]:

# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

# Compute number of train and warmup steps from batch size
num_train_steps = int(len(train_InputExamples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_eval_steps = int(len(dev_InputExamples) / EVAL_BATCH_SIZE)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000

# Converting from TensorFlow Estimator to TPUEstimator
# https://cloud.google.com/tpu/docs/using-estimator-api
def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

In [0]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

model_fn = run_classifier_with_tfhub.model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)

INFO:tensorflow:Using config: {'_model_dir': 'gs://springboard-project-nlp/bert-capstone-project/models/SST-2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.117.190.154:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff6099e54a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.117.190.154:8470', '_evaluation_master': 'grpc://10.117.190.154:8470',

INFO:tensorflow:Using config: {'_model_dir': 'gs://springboard-project-nlp/bert-capstone-project/models/SST-2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.117.190.154:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff6099e54a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.117.190.154:8470', '_evaluation_master': 'grpc://10.117.190.154:8470',

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


# Training the model

In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)

In [0]:
start_time = datetime.now()
print('***** Started training at {} *****'.format(start_time))
print('  Num examples = {}'.format(len(train_InputExamples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
end_time = datetime.now()
print('***** Finished training at {} *****'.format(end_time))
print("  Training took time ", end_time - start_time)

***** Started training at 2019-11-02 00:09:28.135730 *****
  Num examples = 67349
  Batch size = 32
INFO:tensorflow:Skipping training since max_steps has already saved.


INFO:tensorflow:Skipping training since max_steps has already saved.


INFO:tensorflow:training_loop marked as finished


INFO:tensorflow:training_loop marked as finished


***** Finished training at 2019-11-02 00:09:30.228359 *****
  Training took time  0:00:02.092629


In [0]:
dev_input_fn = run_classifier.input_fn_builder(
  features=dev_features,
  seq_length=MAX_SEQ_LENGTH,
  is_training=False,
  drop_remainder=True)

In [0]:
start_time = datetime.now()
print('***** Started evaluation at {} *****'.format(start_time))
print('  Num examples = {}'.format(len(df_dev)))
print('  Batch size = {}'.format(EVAL_BATCH_SIZE))
result = estimator.evaluate(input_fn=dev_input_fn, steps=num_eval_steps)
end_time = datetime.now()
print('***** Finished training at {} *****'.format(end_time))
print("  Training took time ", end_time - start_time)
output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
with tf.gfile.GFile(output_eval_file, "w") as writer:
  print("***** Eval results *****")
  for key in sorted(result.keys()):
    print('  {} = {}'.format(key, str(result[key])))
    writer.write("%s = %s\n" % (key, str(result[key])))

***** Started evaluation at 2019-11-02 00:09:30.258176 *****
  Num examples = 872
  Batch size = 8
INFO:tensorflow:Querying Tensorflow master (grpc://10.117.190.154:8470) for TPU system metadata.


INFO:tensorflow:Querying Tensorflow master (grpc://10.117.190.154:8470) for TPU system metadata.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6313: gs://springboard-project-nlp/bert-capstone-project/models/SST-2/model.ckpt-6313


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6313: gs://springboard-project-nlp/bert-capstone-project/models/SST-2/model.ckpt-6313


INFO:tensorflow:evaluation_loop marked as finished


INFO:tensorflow:evaluation_loop marked as finished


***** Finished training at 2019-11-02 00:10:14.687665 *****
  Training took time  0:00:44.429489
***** Eval results *****
  eval_accuracy = 0.9162844
  eval_loss = 0.49866587
  global_step = 6313
  loss = 0.60559416


# Prediction

In [0]:
# estimator = keras.models.load_model('drive/My Drive/Springboard/projects/data/trained_model.h5')

In [0]:
predict_input_fn = run_classifier.input_fn_builder(
  features=test_features,
  seq_length=MAX_SEQ_LENGTH,
  is_training=False,
  drop_remainder=True)


In [0]:
results = estimator.predict(predict_input_fn)

In [0]:
labels = ["Negative", "Positive"]
predictions = []
for test, result in zip(df_test.values, results):
  label = 0 if result['probabilities'][0] > result['probabilities'][1] else 1
  predictions.append([test[0], labels[label], result['probabilities'], test[1]])

print(predictions[:5])

In [0]:
df_predictions = pd.DataFrame(predictions, columns=['id', 'label', 'probabilities', 'text'])
output_pred_file = os.path.join(OUTPUT_DIR, "pred_results.txt")
with tf.gfile.GFile(output_pred_file, "w") as writer:
  df_predictions.to_csv(writer)

print(df_predictions.head())
print(df_predictions.label.value_counts())

        id     label                probabilities  \
0   810980  Positive  [3.4340774e-05, 0.99996567]   
1  2467161  Positive     [3.22299e-05, 0.9999678]   
2  3572628  Positive   [4.1088944e-05, 0.9999589]   
3  3952097  Positive    [3.413974e-05, 0.9999659]   
4  4377509  Positive    [0.0024238294, 0.9975762]   

                                                text  
0  Miyuki's has been a great host. Helpful and ve...  
1  Nice apartment in one of the best areas of Tok...  
2  A very nice host and place. The location is in...  
3  Miyuki is the most smily face I met when I lan...  
4  The location's wonderful. It's close to Haraju...  
Positive    264454
Negative     33823
Name: label, dtype: int64


In [0]:
with pd.option_context('display.max_colwidth', 500):
  print('<Positive>')
  display(df_predictions.query("label == 'Positive'")[:20])
  print('<Negative>')
  display(df_predictions.query("label == 'Negative'")[:20])

<Positive>


,id,label,probabilities,text
0,810980,Positive,"[3.4340774e-05, 0.99996567]","Miyuki's has been a great host. Helpful and very prompt with all my inquiries.Her place is very neat and tidy. Very accessible to Gaienmae stn ,Harajuku stn and omotesando stn. All within 5-10 mins walking distance.Hope to return soon.Thanks a lot Miyuki!"
1,2467161,Positive,"[3.22299e-05, 0.9999678]","Nice apartment in one of the best areas of Tokyo. Walking distance to Harajuku, Aoyama, Shinjuku and Shibuya, as well as the nearby Ginza line with Gaiemae station.Helpful people to assist you and welcome your stay in the city.Overall a great experience. I would stay here again when in Tokyo."
2,3572628,Positive,"[4.1088944e-05, 0.9999589]",A very nice host and place. The location is incredible for Tokyo (only 5 min walk to Harajuku) and very close to the Subway.I recomend it strongly !
3,3952097,Positive,"[3.413974e-05, 0.9999659]","Miyuki is the most smily face I met when I landed In Tokyo. She offered me a conveniant and confortable house. She was so welcoming and helpful, had a clue about everything, solving each problem with great efficiency and kindness. She's also very funny and I hope i'll have the opportnity to host her someday in Paris."
4,4377509,Positive,"[0.0024238294, 0.9975762]","The location's wonderful. It's close to Harajuku and the famous Takeshita street. Close to 3 different train lines. The housemates are almost never at home so you feel like you have the whole apartment to yourself. The room n toilet is relatively clean although a little messy. There is a supermarket nearby and a 7-11 just on the ground level of the apartment building,"
5,4594739,Positive,"[3.7189337e-05, 0.9999628]","Miyuki was so very helpful and stayed back after hours to wait for me to arrive. The place was comfortable, clean and in the best location, within minutes of Harajuku and a couple of stops from Shibuya. I'd definitely stay here again when I visit Tokyo next. Thank you for such a pleasant stay."
6,6847361,Positive,"[0.0011803653, 0.9988197]","The apartment is an affordable, simple and central 3-bedroom shared flat close to Harajuku. It is very easy to get to anywhere in the city; and even more importantly, if you miss the last train at night, a cab will be affordable or you might even be able to just walk home from areas like Roppongi. There is a 7-Eleven (w/ ATM) right next door and a Natural Lawson around the corner, so you can easily get anything a Kombini can offer 24/7.The bedroom contains a bed, a small desk and a closet wi..."
7,19211289,Positive,"[3.093704e-05, 0.999969]","Thanks Miyuki for a great stay. As described: clean, modest, and very central Great communication"
8,23170075,Positive,"[3.1214087e-05, 0.99996877]",It was great stay. Everything was as described and most importantly Miyuki is very helpful in detail.Location is perfect for any kind of purposes.Definitely recommend this place to anybody who visit Tokyo.
9,27408652,Positive,"[3.530391e-05, 0.9999647]","My first ever solo trip to Tokyo, Japan has been amazing thanks to Miyuki the host! Great great location, around 7 mins to Harajuku train station and Sedagaya train station. Neat and tidy room, quite environment, seven eleven store just around the corner. Highly recommending this room, and I will definitely be staying over at this place or other room listed by Miyuki!"


<Negative>


,id,label,probabilities,text
32,1178861,Negative,"[0.9991842, 0.00081575074]","My friend Kara and I stayed in Yoshimi and Marek's holiday apartment in April (Phone number hidden by Airbnb) The apartement was quite difficult to find even though we had written down directions from (Hidden by Airbnb) maps and asked several people for help. In the end someone from a store was able to find the place on a map and actually walked us to it so we knew where it was. The instructions about finding the key for the door were also not specific enough, so we looked around for a long ..."
87,23319386,Negative,"[0.99740916, 0.0025908754]","Serdecznie polecam Oshiage Holiday Apartment. Mieszkanie wygodne, wyposażone we wszystkie potrzebne sprzęty i bardzo czyste. Gospodarze mili, w razie potrzeby pomocni ale też dyskretni. Okolica cicha i spokojna, około 10 minut spacerem od stacji metra."
89,23997870,Negative,"[0.579505, 0.420495]","房子的地理位置不错,属于闹中取静的地段,距离半藏门线押上站步行大约只要10分钟,洗浴与抽水马桶分开很方便,厨房设备完整,可以自己煮饭然后去附近的小市场买菜吃,很实惠｡缺点是卧室太小,而且只提供两床褥子与被子,询问房东说是可以两个人盖一条被子｡由于我们这次是三个男人一起出行的,导致有一小伙伴只能盖毯子,希望房东能改进,也希望有意租这个房子的人注意｡"
93,29037805,Negative,"[0.9992829, 0.0007171436]",真的很好的住宿體驗物品齊全又乾淨比住宿飯店好一百倍希望能下次再來住對不起房東......小孩有點吵到大家Sorry............
103,41446701,Negative,"[0.5551589, 0.4448411]","非常感谢房东提供的房间,房间距离押上地铁站有点距离,不过按照房东的提示很好找,房间是日式风格的设计,布置的很精巧,浴室和厨房相对比较大,卫生间是独立的,房子虽然不大,但功能齐全,各种生活用品一应具全,厨房里有灶具､厨具冰箱洗衣机等应有尽有,我们自己煮点面吃都很方便｡被子､床单都很干净,特别好的是房东为准备了很多地图和旅游攻略,这让我们觉得很方便!感谢房东贴心周到的考虑,下次来东京我们还会预定这里!"
114,63446964,Negative,"[0.9997124, 0.00028759678]",The host canceled this reservation 65 days before arrival. This is an automated posting.
126,127103100,Negative,"[0.7782606, 0.2217394]","這裡晚上很安靜,一出門就能看見晴空塔,是個很棒的地方住宿的地方在一樓,不用怕東西買多會是回來很累還要爬樓梯｡如果能有攜帶型的分享器那一定會更完美｡"
132,156734085,Negative,"[0.9935434, 0.0064566485]","El alojamiento es típico tradicional japonés , muy bien cuidado, limpio y equipado , con instrucciones muy claras para su total aprovechamiento . Sin embargo es importante considerar que es muy pequeño para cuatro personas , considero que el espacio es adecuado para solo 1 o 2 personas , nosotros fuimos con nuestros hijos y la verdad no cabíamos con lasdonde maletas en la única habitaciónque se tiene para descanso. Tiene lavadora y todo lo indispensable para la cocina."
172,450283939,Negative,"[0.9364574, 0.063542575]",必要なものは､すべて揃っていて家はとても綺麗に掃除されています｡押上駅からは少し離れていますが､慣れれば気にならない距離です｡どこへ行くにも近く､とても便利でした｡お世話になりました｡
222,21332909,Negative,"[0.7545513, 0.24544866]",今回は練馬で研修があるため､近い宿を探していました｡とても便利なﾛｹｰｼｮﾝは想像以上で､ｽｸｰﾙのある場所からは､歩いて15分の距離でした｡宿はとても快適で､しかもいたるところに気配り満載のおもてなしの数々｡まるで我が家のような快適さで､心地よい滞在を過ごすことができました｡WiMAX の使い方がはじめよく分からなかったのですが､とても親切に教えてくださって有難かったです｡洗濯も数回させていただき､気持ちよく過ごせました｡梅干しがとても美味しくて､海外の方でも､きっと気に入るだろうなと感激しました｡とても清潔で居心地満点の宿!また是非､利用したいです｡ありがとうございました‼︎臼井 弘美
